In [249]:
import pandas as pd

In [250]:
cases_path = 'data/cases.csv'

runners_path = 'data/runners.csv'
roles_path = 'data/roles.csv'

assistants_path = 'data/assistants.csv'

ftes_path = 'data/historical_ftes.csv'

In [251]:
df_cases = pd.read_csv(cases_path, sep = ',')

df_runners = pd.read_csv(runners_path, sep = ',')
df_roles = pd.read_csv(roles_path, sep = ',')

df_assistants = pd.read_csv(assistants_path, sep = ',')
df_ftes = pd.read_csv(ftes_path, sep = ',')

In [252]:
support_groups = ['Implementacion de Servicios Estrategicos CedEx Soporte',
                  'Operaciones CedEx Implementacion de Servicios Estrategicos CedEx Soporte Enterdev',
                  'Operaciones CedEx Soporte en Campo UNISYS']

resolved_states = ['Resuelto', 'Cerrado']

In [253]:
df_cases = (df_cases.query("Grupo in @support_groups")
                    .query("Estado in @resolved_states")
                    .query("Resumen.str.startswith('RPA')")
                    .query("Sintoma.str.startswith('ACIS')"))

df_cases['assistant_nickname'] = df_cases['Resumen'].str.split(' - ').str[0].str.upper()

df_cases['Sintoma'] = df_cases['Sintoma'].str.replace('?', '', regex = False)
df_cases['symptom_category'] = df_cases['Sintoma'].str.split(' - ').str[1]

In [254]:
df_runners = df_runners.dropna(subset = ['machine'])
df_runners['machine'] = df_runners['machine'].str.upper()

In [255]:
df_runners = pd.merge(df_runners[['role_id', 'machine']], df_roles[['role_id', 'assistant_id']], on = 'role_id', how = 'inner')

In [256]:
df_runners = df_runners.groupby(by = 'assistant_id').agg({'machine': 'nunique'}).reset_index()

In [257]:
df_assistants = (df_assistants.query("assistant_type == 'RPA'")
                                .query("assistant_state == 'Activo'"))

df_assistants['assistant_nickname'] = df_assistants['assistant_nickname'].str.upper()

In [258]:
df_assistants = pd.merge(df_assistants[['assistant_id', 'assistant_nickname', 'assistant_bia']], df_runners, on = 'assistant_id', how = 'inner')

In [259]:
df_ftes = (df_ftes.query("anio == 2022")
                    .query("mes in (10, 11, 12)"))

df_ftes = df_ftes.groupby(by = 'id_componente').agg({'ftes': 'mean'}).reset_index()

In [260]:
df_assistants = pd.merge(df_assistants, df_ftes, left_on = 'assistant_id', right_on = 'id_componente', how = 'inner')

In [261]:
df_cases = pd.merge(df_cases, df_assistants[['assistant_nickname', 'assistant_bia', 'machine', 'ftes']], on = 'assistant_nickname', how = 'inner')

In [262]:
df_cases

,Nro Caso,Usuario final afectado,Estado,Asignatario,Grupo,Urgencia,Sintoma,Resumen,Medio_ingreso,Fecha Apertura,Fecha Resuelto,assistant_nickname,symptom_category,assistant_bia,machine,ftes
0,40001218,Santiago Ramirez Giraldo,Resuelto,Luis Diego Cordoba Quinto,Operaciones CedEx Soporte en Campo UNISYS,Media,ACIS - Software - Conexión remota y reinicio,RPAGPC - Reinicio máquina,Autoreporte,20/02/2023,20/02/2023,RPAGPC,Software,NO,20,0.541000
1,40001204,Santiago Ramirez Giraldo,Resuelto,Luis Diego Cordoba Quinto,Operaciones CedEx Soporte en Campo UNISYS,Media,ACIS - Software - Conexión remota y reinicio,RPAGPC - Reinicio maquina,Autoreporte,20/02/2023,20/02/2023,RPAGPC,Software,NO,20,0.541000
2,39992967,Santiago Ramirez Giraldo,Resuelto,Luis Diego Cordoba Quinto,Operaciones CedEx Soporte en Campo UNISYS,Media,ACIS - Software - Conexión remota y reinicio,RPAGPC - Reinicio maquina,Autoreporte,17/02/2023,17/02/2023,RPAGPC,Software,NO,20,0.541000
3,39988860,Santiago Ramirez Giraldo,Resuelto,Luis Diego Cordoba Quinto,Operaciones CedEx Soporte en Campo UNISYS,Media,ACIS - Software - Conexión remota y reinicio,RPAGPC - Reinicio maquina,Autoreporte,16/02/2023,16/02/2023,RPAGPC,Software,NO,20,0.541000
4,39974312,Eduardo Jose Freay Santamaria,Cerrado,Luis Diego Cordoba Quinto,Operaciones CedEx Soporte en Campo UNISYS,Media,ACIS - Software - Conexión remota y reinicio,RPAGPC - Conexión Remota,Autoreporte,13/02/2023,13/02/2023,RPAGPC,Software,NO,20,0.541000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2963,20738580,Jhon Fredy Rincon Gil,Cerrado,Julian David Loaiza Arango,Implementacion de Servicios Estrategicos CedEx...,Media,ACIS - Gestión - Pendientes usuario disponibil...,RPADIA - TCS - Revisar flujo de Transaccion,Autoreporte,7/07/2022,15/07/2022,RPADIA,Gestión,NO,7,0.000000
2964,20761817,Jehison Orlando Restrepo Valencia,Cerrado,Raul Eduardo Arango Saldarriaga,Operaciones CedEx Soporte en Campo UNISYS,Media,ACIS - Software - Instalación/Reinstalación de...,RPAPUC - Reinstalación AA,Autoreporte,13/07/2022,13/07/2022,RPAPUC,Software,NO,2,0.015133
2965,20756506,Daniel Ramirez Vergara,Cerrado,Julian David Lopez Granda,Operaciones CedEx Soporte en Campo UNISYS,Media,ACIS - Hardware - Caída del servidor de bases ...,RPAPYP - Validar conexión,Autoreporte,12/07/2022,13/07/2022,RPAPYP,Hardware,NO,1,0.000000
2966,20740574,Daniel Ramirez Vergara,Cerrado,Raul Eduardo Arango Saldarriaga,Operaciones CedEx Soporte en Campo UNISYS,Media,ACIS - Software - Conexión remota y reinicio,RPAPYP - Conexión Remota,Autoreporte,8/07/2022,8/07/2022,RPAPYP,Software,NO,1,0.000000
